# Tracking XGBoost models using Verta

Verta's experiment management system enables data scientists to track rich information about their modeling experiments including data such as metrics, hyperparameters, confusion matrices, examples of input and output data, and many others.

This notebook shows how to use Verta's experiment management system with models developed in XGBoost. See Verta [documentation](https://docs.verta.ai/verta/experiment-management) for full details on Verta's experiment management capabilities.

Updated for Verta version: 0.18.2

This example features:
- **XGBoost**'s native API for cross-validating and training gradient-boosted trees
- **scikit-learn**'s `ParameterGrid` utility for iterating over a hyperparameter grid
- **verta**'s Python client integrated into the grid search loop
- **verta**'s Python client retrieving the best run from the grid search to calculate and log full training accuracy

<a href="https://colab.research.google.com/github/VertaAI/modeldb/blob/master/client/workflows/examples/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Imports

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import datasets
from sklearn import model_selection

import xgboost as xgb

### 0.1 Verta import and setup

In [2]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [3]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST']

In [4]:
from verta import Client
from verta.utils import ModelAPI
import os

client = Client(os.environ['VERTA_HOST'])

---

## 1. Model Training

### 1.1 Prepare Data

In [5]:
data = datasets.load_wine()

X = data['data']
y = data['target']
labels = data['target_names'].tolist()

dtrain = xgb.DMatrix(X, label=y)

In [6]:
df = pd.DataFrame(np.hstack((X, y.reshape(-1, 1))),
                  columns=data['feature_names'] + ['species'])

df.head()

### 1.2 Prepare Hyperparameters

In [7]:
grid = model_selection.ParameterGrid({
    'eta': [0.5, 0.7],
    'max_depth': [1, 2, 3],
    'num_class': [10],
})

### 1.3 Train model

In [8]:
proj = client.set_project("Wine Classification")
expt = client.set_experiment("Boosted Trees")

def run_experiment(hyperparams):
    run = client.set_experiment_run()
    
    # log hyperparameters
    run.log_hyperparameters(hyperparams)
    
    # run cross validation on hyperparameters
    cv_history = xgb.cv(hyperparams, dtrain,
                        nfold=5,
                        metrics=("merror", "mlogloss"))

    # log observations from each iteration
    for _, iteration in cv_history.iterrows():
        for obs, val in iteration.iteritems():
            run.log_observation(obs, val)
            
    # log error from final iteration
    final_val_error = iteration['test-merror-mean']
    run.log_metric("val_error", final_val_error)
    print("{} Mean error: {:.4f}".format(hyperparams, final_val_error))
    
# NOTE: run_experiment() could also be defined in a module, and executed in parallel
for hyperparams in grid:
    run_experiment(hyperparams)

---

## 2. Retrieve logged metadata

In [9]:
best_run = expt.expt_runs.sort("metrics.val_error", descending=False)[0]
print("Validation Error: {:.4f}".format(best_run.get_metric("val_error")))

best_hyperparams = best_run.get_hyperparameters()
print("Hyperparameters: {}".format(best_hyperparams))

## 3. Train on Full Dataset using best hyperparams

In [10]:
model = xgb.XGBClassifier(**best_hyperparams)
model.fit(X, y)

In [11]:
train_acc = model.score(X, y)
best_run.log_metric("full_train_acc", train_acc)
best_run.log_tag("best-run")

# compute and log confusion matrix
from sklearn.metrics import confusion_matrix
from verta.data_types import ConfusionMatrix
confusion_matrix = confusion_matrix(y, model.predict(X))
best_run.log_attribute("confusion_matrix", ConfusionMatrix(confusion_matrix.tolist(), labels))


# log additional info for reproducibility
from verta.code import Notebook
from verta.environment import Python
best_run.log_artifact("notebook", "xgboost.ipynb")
best_run.log_environment(Python(["xgboost", "sklearn", "numpy", "pandas"]))
best_run.log_artifact("data", df)

print("Training accuracy: {:.4f}".format(train_acc))

In [12]:
best_run

---